In [ ]:
import numpy as np

from scipy.integrate import solve_ivp
from scipy.linalg import null_space

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

import warnings

# Thermal explosion

In the first PC, we have considered the dynamical system modelint thermal explosion in the large activation energy and large heat of reaction limit:

$$
{\mathrm d}_\tau \widetilde\theta = e^{\widetilde\theta} -  \alpha_0\,\widetilde\theta.
$$

For the purpose of studying the branches of equilibrium points and limit points, we rather consider the following
form of the system:

$$
{\mathrm d}_t \theta = {\mathrm F}_{\mathrm k} \, e^{\theta} - \theta
$$

where the Frank-Kamemetskii parameter ${\mathrm F}_{\mathrm k} = 1/\alpha_0$ and which is obtained by a simple change of the temporal scale. The ${\mathrm F}_{\mathrm k}$ parameter is here taken as the bifurcation parameter.

In [ ]:
def f(y):
    fk, theta = y
    return np.array([fk * np.exp(theta) - theta])

# specific form of f to use solve_ivp method
def f2(t, y, fk):
    return f((fk, y))

def df(y):
    fk, theta = y
    return np.array([[fk * np.exp(theta) - 1]])

def df_aug(y) :
    fk, theta = y
    return np.matrix(np.array([np.exp(theta), fk * np.exp(theta) - 1]))

In [ ]:
fk_lim = 1/np.exp(1)  

fk = 0.1

tini = 0. 
tend = 10.

yini = (0.,)

tol = 1e-10
sol = solve_ivp(f2, (tini, tend), yini, method="RK45", args=(fk,), rtol=tol, atol=tol)

fig = go.Figure()

fig.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name='theta'))

steps = []
for fki in np.arange(0.1, fk_lim, 0.05):
    sol = solve_ivp(f2, (tini, tend), yini, method="RK45", args=(fki,), rtol=tol, atol=tol)
    args = [{"x": [sol.t], "y": [sol.y[0]]}]
    step = dict(method="update", label = f"{fki:.2f}", args=args)
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'Fk = '}, steps=steps)]

fig.update_layout(sliders=sliders)
fig.update_xaxes(title_text="t")
fig.update_yaxes(title_text="theta")

fig['layout']['sliders'][0]['pad']=dict(t=50)

fig.show()

## Natural continuation

In [ ]:
def nat_cont_trivial_pred(sol_ini, npts, dlamb, f, df, tol=1.e-8, nmax_newton=100):
        
    print("Natural continuation (trivial prediction)")
            
    sol = np.zeros((len(sol_ini), npts))
    sol[:,0] = sol_ini
    
    is_stable = np.zeros(npts, dtype=bool)
    eig_vals_eq = np.linalg.eigvals(df(sol_ini))
    is_stable[0] = np.all(np.real(eig_vals_eq) < 0)
    
    nb_iter_newton = 0
        
    for it in range(npts-1):
               
        # Newton iteration
        soln = np.copy(sol[:, it])
        soln[0] = soln[0] + dlamb
        #print("Iter nb ", 0, " ||f(y)|| = ",  np.linalg.norm(f(soln)))
        for it_newton in range(nmax_newton):
            soln[1:] = soln[1:] + np.linalg.solve(df(soln), -f(soln))
            #print("Iter nb ", it_newton+1, " ||f(y)|| = ",  np.linalg.norm(f(soln)))
            
            if (np.linalg.norm(f(soln)) < tol):
                nb_iter_newton += it_newton+1
                #print(it_newton+1)
                break
                
        if (np.linalg.norm(f(soln)) > tol):
            warnings.warn(f"Newton may not have converged")
            return sol[:,:it+1], is_stable[:it+1]
        
        eig_vals = np.linalg.eigvals(df(soln))
        is_stable[it+1] = np.all(np.real(eig_vals)<0)
        sol[:, it+1] = soln
        
    print(f"Average number of newton iterations per step : {nb_iter_newton/(npts-1)}")
        
    return sol, is_stable

In [ ]:
sol_ini = np.array([0.,0.])
npts = 100
dlamb = 1e-2
sol, is_stable = nat_cont_trivial_pred(sol_ini, npts, dlamb, f, df)

fig = go.Figure()
marker_unstab=dict(symbol='x', color='rgb(196,78,82)')
marker_stab=dict(symbol='x', color='rgb(85,168,104)')
fig.add_trace(go.Scatter(x=sol[0, is_stable], y=sol[1, is_stable], name='stable sol.', mode='markers', marker=marker_stab, showlegend=True))
fig.add_trace(go.Scatter(x=sol[0, np.logical_not(is_stable)], y=sol[1, np.logical_not(is_stable)], name='unstable sol.', mode='markers', marker=marker_unstab))
fig.update_xaxes(title_text="Fk")
fig.update_yaxes(title_text="theta")
fig.show()

## Pseudo arc-length

In [ ]:
def pseudo_arclength_cont(sol_ini, vect_ini, npts, ds, f, df, tol=1.e-8, nmax_newton=100):
                
    sol = np.zeros((len(sol_ini), npts))
    sol[:,0] = sol_ini
    
    is_stable = np.empty(npts, dtype=bool)
    eig_vals_eq = np.linalg.eigvals(df(sol_ini)[:,1:])
    is_stable[0] = np.all(np.real(eig_vals_eq) < 0)
            
    vect = vect_ini
    sol_n = sol_ini
    
    for inpts in range(npts-1):
        sol_pred = sol_n + ds * vect
        # Newton iteration
        sol_n = sol_pred
        res = np.concatenate((np.array([0]),f(sol_n)))
        for it_newton in range(nmax_newton):
            jac = np.block([[vect],[df(sol_n)]])
            sol_n += np.linalg.solve(jac, -res)
            res = np.concatenate((np.array([np.dot(sol_n-sol_pred,vect)]), f(sol_n)))
            err = np.sum(res)
            if (np.sum(res) < tol):
                break
        if (np.sum(res)>tol) :
            warnings.warn("Newton may not have converged")
            print(err)
            
        sol[:,inpts+1] = sol_n
        
        eig_vals_eq = np.linalg.eigvals(df(sol_n)[:,1:])
        is_stable[inpts+1] = np.all(np.real(eig_vals_eq) < 0)
            
        vect_old = np.copy(vect)
        
        vect = null_space(df(sol_n))[:,0] ## computation of a new tangent vector at the new point 
        
        if np.dot(vect, vect_old) < 0: ## checking that we still go in the same direction
            vect = -vect
                            
    return sol, is_stable

In [ ]:
yeq_ini = np.array([0.])
lamb_ini = 0.
sol_ini = np.array([0.,0.])
vect_ini = np.array([1.,0.])
npts = 1000
ds = 1e-2
sol, is_stable = pseudo_arclength_cont(sol_ini, vect_ini, npts, ds, f, df_aug)

fig = go.Figure()
marker_unstab=dict(symbol='x', color='rgb(196,78,82)')
marker_stab=dict(symbol='x', color='rgb(85,168,104)')
fig.add_trace(go.Scatter(x=sol[0, is_stable], y=sol[1, is_stable], name='stable sol.', mode='markers', marker=marker_stab))
fig.add_trace(go.Scatter(x=sol[0, np.logical_not(is_stable)], y=sol[1, np.logical_not(is_stable)], name='unstable sol.', mode='markers', marker=marker_unstab))
fig.update_xaxes(title_text="Fk")
fig.update_yaxes(title_text="theta")
fig.show()